In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Create a text corpus ( run only once )

#### Save a copy of the dataset on the drive

In [ ]:
# Method 3
# kaggle with colab
!pip install kaggle 

In [ ]:
#before importing the dataset we want to use this code
# The Kaggle API client expects this file to be in ~/.kaggle,
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c jigsaw-unintended-bias-in-toxicity-classification

  0% 0.00/224k [00:00<?, ?B/s]
100% 224k/224k [00:00<00:00, 33.0MB/s]
 97% 268M/276M [00:03<00:00, 81.0MB/s]
100% 276M/276M [00:03<00:00, 93.8MB/s]
 96% 62.0M/64.7M [00:00<00:00, 62.2MB/s]
100% 64.7M/64.7M [00:00<00:00, 68.0MB/s]
 97% 315M/326M [00:04<00:00, 87.6MB/s]
100% 326M/326M [00:04<00:00, 74.2MB/s]
 32% 5.00M/15.8M [00:00<00:00, 25.3MB/s]
100% 15.8M/15.8M [00:00<00:00, 61.8MB/s]
 41% 5.00M/12.3M [00:00<00:00, 26.7MB/s]
100% 12.3M/12.3M [00:00<00:00, 48.7MB/s]
 41% 5.00M/12.1M [00:00<00:00, 21.7MB/s]
100% 12.1M/12.1M [00:00<00:00, 40.5MB/s]
 32% 5.00M/15.9M [00:00<00:00, 28.1MB/s]
100% 15.9M/15.9M [00:00<00:00, 52.9MB/s]


In [ ]:
!unzip all_data.csv.zip

Archive:  all_data.csv.zip
  inflating: all_data.csv            


In [ ]:
!cp all_data.csv drive/MyDrive/toxic_span_identification/pre_training/civil_comments_all_data.csv

#### Preprocess the civil comments dataset

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("./drive/MyDrive/toxic_span_identification/pre_training/civil_comments_all_data.csv")
df.head()

,id,comment_text,split,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,toxicity,severe_toxicity,obscene,sexual_explicit,identity_attack,insult,threat,male,female,transgender,other_gender,heterosexual,homosexual_gay_or_lesbian,bisexual,other_sexual_orientation,christian,jewish,muslim,hindu,buddhist,atheist,other_religion,black,white,asian,latino,other_race_or_ethnicity,physical_disability,intellectual_or_learning_disability,psychiatric_or_mental_illness,other_disability,identity_annotator_count,toxicity_annotator_count
0,1083994,He got his money... now he lies in wait till a...,train,2017-03-06 15:21:53.675241+00,21,NaN,317120,approved,0,0,0,2,0,0.373134,0.044776,0.089552,0.014925,0.000000,0.343284,0.014925,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,67
1,650904,Mad dog will surely put the liberals in mental...,train,2016-12-02 16:44:21.329535+00,21,NaN,154086,approved,0,0,1,2,0,0.605263,0.013158,0.065789,0.013158,0.092105,0.565789,0.065789,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,76
2,5902188,And Trump continues his lifelong cowardice by ...,train,2017-09-05 19:05:32.341360+00,55,NaN,374342,approved,1,0,2,3,7,0.666667,0.015873,0.031746,0.000000,0.047619,0.666667,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,63
3,7084460,"""while arresting a man for resisting arrest"".\...",test,2016-11-01 16:53:33.561631+00,13,NaN,149218,approved,0,0,0,0,0,0.815789,0.065789,0.552632,0.592105,0.000000,0.684211,0.105263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,76
4,5410943,Tucker and Paul are both total bad ass mofo's.,train,2017-06-14 05:08:21.997315+00,21,NaN,344096,approved,0,0,0,1,0,0.550000,0.037500,0.337500,0.275000,0.037500,0.487500,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,80


In [ ]:
comments = list(df['comment_text'])

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize
import re
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
comments[:10]

["He got his money... now he lies in wait till after the election in 2 yrs.... dirty politicians need to be afraid of Tar and feathers again... but they aren't and so the people get screwed.",
 'Mad dog will surely put the liberals in mental hospitals. Boorah',
 'And Trump continues his lifelong cowardice by not making this announcement himself.\n\nWhat an awful human being .....',
 '"while arresting a man for resisting arrest".\n\nIf you cop-suckers can\'t see a problem with this, then go suck the barrel of a Glock.',
 "Tucker and Paul are both total bad ass mofo's.",
 "Cry me a river, why don't you.\nDrinking, drugs, dressing like a ho.  Bad recipe for disaster!",
 'I really wish the DP would bounce people trying to make a moral equivalence between Nazis and BLM. \n\n\nThat kind of hateful BothSiderist garbage had got to stop. It\'s not even funny any more. \n\n\nActual Nazis are marching in this country and killing people. And the response of some is "but the blahs". And the DP allo

In [ ]:
import string

In [ ]:
with open("./drive/MyDrive/toxic_span_identification/pre_training/civil_comments_text_corpus.txt","w") as myfile:
  for comment in comments:
    if str(comment) == "nan":
      continue
    sentences = sent_tokenize(comment)
    for sent in sentences:
      formatted_sent = sent.strip()
      formatted_sent = re.sub('[\s]*\n[\s]*','\n',formatted_sent)
      formatted_sent = re.sub('[ ]+',' ',formatted_sent)
      l = len(formatted_sent)
      if l == 0 or l == 1 :
        continue
      myfile.write(formatted_sent)
      myfile.write("\n")

#### Random sample of text corpus

In [ ]:
!cp /content/drive/MyDrive/toxic_span_identification/pre_training/civil_comments_text_corpus.txt civil_comments_text_corpus.txt

In [ ]:
with open("civil_comments_text_corpus.txt","r") as f:
  sentences = f.readlines()
print(len(sentences))

7317904


In [ ]:
import random

In [ ]:
sample_sentences = random.sample(sentences, 3200000)

In [ ]:
with open("civil_comments_text_corpus_reduced.txt","w") as f:
  for sent in sample_sentences:
    f.write(sent)

In [ ]:
!cp civil_comments_text_corpus_reduced.txt /content/drive/MyDrive/toxic_span_identification/pre_training/civil_comments_text_corpus_reduced.txt

### Train a language model

In [ ]:
!pip uninstall -y tensorflow

Uninstalling tensorflow-2.4.0:
  Successfully uninstalled tensorflow-2.4.0


In [ ]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 1.5MB 17.7MB/s 
     |████████████████████████████████| 2.9MB 51.8MB/s 
     |████████████████████████████████| 890kB 45.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=f18438b42781f5a2eb4c95c543833374bdb016132dbc796bacc3200b9b933c99
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 163kB 22.7MB/s 
     |████████████████████████████████| 17.7MB 217kB/s 
     |████████████████████████████████| 245kB 57.0MB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


In [ ]:
!cp drive/MyDrive/toxic_span_identification/pre_training/civil_comments_text_corpus_reduced.txt civil_comments_text_corpus_reduced.txt 

In [ ]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")

In [ ]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM.from_pretrained("./drive/MyDrive/toxic_span_identification/pre_training_roberta_base_checkpoints/checkpoint-260000")
model.config
model.num_parameters()

124697433

In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset("text", data_files = "civil_comments_text_corpus_reduced.txt")
dataset['train']

Using custom data configuration default


Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-dddb0beab75bf66d/0.0.0/daf90a707a433ac193b369c8cc1772139bb6cca21a9c7fe83bdd16aad9b9b6ab. Subsequent calls will reuse this data.


Dataset({
    features: ['text'],
    num_rows: 3200022
})

In [ ]:
def encode(sample):
  return tokenizer(sample['text'], add_special_tokens=True, truncation=True, max_length=128)
dataset = dataset.map(encode, batched=True, batch_size = 512)
dataset.set_format(type='torch', columns=['input_ids','attention_mask'])

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir = "./drive/MyDrive/toxic_span_identification/pre_training_roberta_base_checkpoints",
    overwrite_output_dir = True,
    num_train_epochs = 3,
    per_gpu_train_batch_size = 32,
    save_steps = 10_000,
    learning_rate = 2e-5,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset['train'],
)

In [ ]:
# trainer.train()

In [ ]:
%%time
trainer.train("./drive/MyDrive/toxic_span_identification/pre_training_roberta_base_checkpoints/checkpoint-260000")

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
/usr/local/lib/python3.6/dist-packages/datasets/arrow_dataset.py:850: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.tensor(x,

Step,Training Loss
260500,0.003810
261000,1.987712
261500,1.980686
262000,1.986991
262500,1.992656
263000,2.010785
263500,1.996902
264000,1.977644
264500,1.984752
265000,1.999360


Buffered data was truncated after reaching the output size limit.